<a href="https://colab.research.google.com/github/Andresf-Asprilla/Modelo-segmentacion-APCIVMAPCAs/blob/main/Evaluaci%C3%B3n_del_HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Antes de ejecutar el siguiente script, debe instalar las siguientes dependencias:
```
!pip install git+https://github.com/Project-MONAI/MONAI#egg=monai

!pip install nnunetv2
```



Nota: Durante la instalación de estos frameworks puede generarse un mensaje de advertencia; este puede ser ignorado, ya que no afecta los resultados ni el funcionamiento del proceso de evaluación.

#Ejecuta el siguiente código una sola vez haciendo clic dentro de la celda y luego presionando Control + Enter.

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

import os
import numpy as np
import torch
from monai.metrics import DiceMetric, HausdorffDistanceMetric
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO

def region_or_label_to_mask(segmentation: np.ndarray, region_or_label):
    if np.isscalar(region_or_label):
        return segmentation == region_or_label
    mask = np.zeros_like(segmentation, dtype=bool)
    for r in region_or_label:
        mask[segmentation == r] = True
    return mask

def compute_tp_fp_fn_tn(mask_ref, mask_pred, ignore_mask=None):
    if ignore_mask is None:
        use_mask = np.ones_like(mask_ref, dtype=bool)
    else:
        use_mask = ~ignore_mask
    tp = np.sum((mask_ref & mask_pred) & use_mask)
    fp = np.sum(((~mask_ref) & mask_pred) & use_mask)
    fn = np.sum((mask_ref & (~mask_pred)) & use_mask)
    tn = np.sum(((~mask_ref) & (~mask_pred)) & use_mask)
    return tp, fp, fn, tn


def to_monai_tensor(np_array):
    tensor = torch.from_numpy(np_array.astype(np.uint8)).to(torch.float32)
    tensor = tensor.unsqueeze(0)
    return tensor


def compute_metrics(reference_file, prediction_file, image_reader_writer, labels, ignore_label=None):
    seg_ref, _ = image_reader_writer.read_seg(reference_file)
    seg_pred, _ = image_reader_writer.read_seg(prediction_file)

    ignore_mask = seg_ref == ignore_label if ignore_label is not None else None
    #dice_metric = DiceMetric(include_background=False, reduction="mean")
    hd_metric = HausdorffDistanceMetric(include_background=False, percentile=95.0, reduction="mean")

    results = {
        'reference_file': reference_file,
        'prediction_file': prediction_file,
        'metrics': {}
    }

    for label in labels:
        mask_ref = region_or_label_to_mask(seg_ref, label)
        mask_pred = region_or_label_to_mask(seg_pred, label)

        ref_tensor = to_monai_tensor(mask_ref)
        pred_tensor = to_monai_tensor(mask_pred)
        #dice_monai = dice_metric(pred_tensor, ref_tensor).item()
        if mask_ref.sum() > 0 and mask_pred.sum() > 0:
            hd_score = hd_metric(pred_tensor, ref_tensor).item()
        else:
            hd_score = np.nan

        tp, fp, fn, tn = compute_tp_fp_fn_tn(mask_ref, mask_pred, ignore_mask)

        metrics = {
            #'MONAI_DiceMean': float(dice_monai),
            'Dice': float(np.nan if tp + fp + fn == 0 else 2 * tp / (2 * tp + fp + fn)),
            'HD95': float(hd_score),
            'IoU': float(np.nan if tp + fp + fn == 0 else tp / (tp + fp + fn)),
            'TP': int(tp),
            'FP': int(fp),
            'FN': int(fn),
            'TN': int(tn),
            'n_pred': int(fp + tp),
            'n_ref': int(fn + tp),
        }
        results['metrics'][label] = metrics

    return results

def evaluate_folder_niigz(folder_ref, folder_pred, labels, output_file="summary2.json",
                          ignore_label=None):
    file_ending = ".nii.gz"
    reader_writer = SimpleITKIO()

    files_ref_set = set(subfiles(folder_ref, suffix=file_ending, join=False))
    files_pred_set = set(subfiles(folder_pred, suffix=file_ending, join=False))


    common_files = sorted(list(files_ref_set & files_pred_set))
    if not common_files:
        raise RuntimeError("No hay archivos comunes entre referencia y predicción.")

    files_ref = [join(folder_ref, f) for f in common_files]
    files_pred = [join(folder_pred, f) for f in common_files]

    results = []
    for ref, pred in zip(files_ref, files_pred):
        results.append(compute_metrics(ref, pred, reader_writer, labels, ignore_label))

    metric_list = list(results[0]['metrics'][labels[0]].keys())
    means = {l: {m: np.nanmean([r['metrics'][l][m] for r in results]) for m in metric_list} for l in labels}
    foreground_mean = {m: np.mean([means[l][m] for l in labels if l != 0]) for m in metric_list}

    summary = {'metric_per_case': results, 'mean': means, 'foreground_mean': foreground_mean}
    save_json(summary, output_file)
    return summary


Para realizar la evaluacion de la metrica HD, ejecute la siguiente: haga clic dentro de la celda y luego presione Control + Enter . Sin embargo, asegúrese de modificar las rutas de los directorios para indicar tanto la ubicación de las imágenes como el lugar donde desea que se almacenen los archivos convertidos.

In [ ]:
folder_ref = "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset506_APCIVMP/labelsTr"
folder_pred = "/content/drive/MyDrive/Colab Notebooks/nnUNet_results/Dataset506_APCIVMP/nnUNetTrainer_10epochs__nnUNetPlans__3d_lowres/fold_0/validation"
labels = [1, 2, 3, 4]

summary = evaluate_folder_niigz(
    folder_ref=folder_ref,
    folder_pred=folder_pred,
    labels=labels,
    output_file="/content/drive/MyDrive/Colab Notebooks/nnUNet_results/Dataset506_APCIVMP/nnUNetTrainer_10epochs__nnUNetPlans__3d_lowres/fold_0/validation/summary.json"
)

print("✅ Evaluación completa")
